In [1]:
from datasets import load_dataset
import pandas as pd
from rank_bm25 import BM25Okapi
from tqdm import tqdm

## Loading datasets: 

In [2]:
eval_queries_df = pd.read_parquet('sample_retrieval_dataset.parquet')
eval_queries = set(eval_queries_df['query'])
print(len(eval_queries))
eval_queries_df.head()

494


,query,parent_asin
0,plastic chopper boards,B01DIPJQC6
1,women exercise clothing,B01KGZHOGQ
2,mobarel adjustable wrench,B00D2SXJ0K
3,air filter for car,B00854L3YU
4,construx lego,B078J4C7D6


In [3]:
dataset = load_dataset("AdhirajSingh1206/TREC-Zero-Shot-Product-Search", split="full")
dataset = dataset.to_pandas()
print(len(dataset))
dataset.head()

Resolving data files:   0%|          | 0/21 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/21 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/21 [00:00<?, ?it/s]

3289158


,main_category,title,average_rating,rating_number,features,description,images,videos,details,parent_asin
0,Pet Supplies,Collapsible Dog Travel Bowl - Large,1.0,1,['Holds both food and water' 'Easy to clean an...,['Collapsible Dog Travel Large Bowl. This very...,"{'hi_res': array([None], dtype=object), 'large...","{'title': array([], dtype=object), 'url': arra...","{""Target Species"": ""Dog"", ""Special Feature"": ""...",B005FRLGF6
1,Buy a Kindle,ThoughtFull: Discovering Unique Gifts in Each ...,4.8,105,['This encouraging story shows how life change...,['About the Author' 'For more than twenty-two ...,"{'hi_res': array([None], dtype=object), 'large...","{'title': array([], dtype=object), 'url': arra...","{""Publisher"": ""B&H Kids (August 15, 2018)"", ""P...",B07FYWV1HZ
2,Toys & Games,YU-GI-OH! - Brain Control (TLM-EN038) - The Lo...,4.7,4,['A single individual card from the Yu-Gi-Oh! ...,"[""Yu-Gi-Oh! is a strategic trading card game i...","{'hi_res': array([None], dtype=object), 'large...","{'title': array([], dtype=object), 'url': arra...","{""Item Weight"": ""0.06 ounces"", ""Manufacturer r...",B0028WKKU8
3,Appstore for Android,Word Search Inspiration,4.3,218.0,['EASY TO PLAY - It starts easy but gets chall...,['Find inspiration and activate your brain wit...,"{'hi_res': array([None, None, None, None, None...","{'title': array([''], dtype=object), 'url': ar...","{""Release Date"": ""2020"", ""Date first listed on...",B088R6YNZX
4,Prime Video,Ancient Greek Civilization Lecture 3 of 10 The...,4.2,7.0,['1 h 19 min' '2010' 'NR'],['Lecture 3 of 10: The Trojan War. As the My...,"{'hi_res': array([None], dtype=object), 'large...","{'title': array([], dtype=object), 'url': arra...","{""Content advisory"": [""Nudity"", ""violence""], ""...",B0042QES9M


## Creating simple baseline using bm25 on the title:

In [4]:
queries = list(eval_queries_df['query'])
ideal_item_ids = list(eval_queries_df['parent_asin'])

hit_rates = []

tokenized_titles = [title.lower().split() for title in dataset['title']]
bm25 = BM25Okapi(tokenized_titles)

In [5]:
for query, ideal_id in tqdm(zip(queries, ideal_item_ids)):
    
    query_tokens = query.lower().split()
    bm25_scores = bm25.get_scores(query_tokens)

    sorted_product_ids = [product_id for _, product_id in sorted(zip(bm25_scores, dataset['parent_asin']), reverse=True)]

    if ideal_id in sorted_product_ids[:50]:
        hit_rates.append(1)
    else:
        hit_rates.append(0)

average_hit_rate = sum(hit_rates) / len(hit_rates)
print(f"Average Hit Rate @ 50: {average_hit_rate}")

500it [2:33:33, 18.43s/it]

Average Hit Rate @ 50: 0.236
